In [2]:
import requests
import json
import pandas as pd
import pymysql.cursors
from sqlalchemy import create_engine

Get number of match days so far

In [3]:
url = "https://heisenbug-premier-league-live-scores-v1.p.rapidapi.com/api/premierleague/table"

headers = {
    'x-rapidapi-host': "heisenbug-premier-league-live-scores-v1.p.rapidapi.com",
    'x-rapidapi-key': "58ba862574msh2d22f18a5692fefp1e9a03jsn31da5fc820dc"
    }

league_table = json.loads(requests.request("GET", url, headers=headers).text)

print(league_table)

{'records': [{'team': 'Liverpool', 'played': 36, 'win': 30, 'draw': 3, 'loss': 3, 'goalsFor': 77, 'goalsAgainst': 29, 'points': 93}, {'team': 'Manchester City', 'played': 37, 'win': 25, 'draw': 3, 'loss': 9, 'goalsFor': 97, 'goalsAgainst': 35, 'points': 78}, {'team': 'Chelsea', 'played': 36, 'win': 19, 'draw': 6, 'loss': 11, 'goalsFor': 64, 'goalsAgainst': 49, 'points': 63}, {'team': 'Manchester United', 'played': 36, 'win': 17, 'draw': 11, 'loss': 8, 'goalsFor': 63, 'goalsAgainst': 35, 'points': 62}, {'team': 'Leicester', 'played': 37, 'win': 18, 'draw': 8, 'loss': 11, 'goalsFor': 67, 'goalsAgainst': 39, 'points': 62}, {'team': 'Wolverhampton Wanderers', 'played': 37, 'win': 15, 'draw': 14, 'loss': 8, 'goalsFor': 51, 'goalsAgainst': 38, 'points': 59}, {'team': 'Tottenham', 'played': 37, 'win': 16, 'draw': 10, 'loss': 11, 'goalsFor': 60, 'goalsAgainst': 46, 'points': 58}, {'team': 'Burnley', 'played': 37, 'win': 15, 'draw': 9, 'loss': 13, 'goalsFor': 42, 'goalsAgainst': 48, 'points': 5

In [4]:
max_matches = 0
for i in league_table['records']:
    if i['played'] > max_matches:
        max_matches = i['played']

In [5]:
max_matches

37

Now that we know the number of matchdays I will make a dataframe with every match that has been played

In [6]:
url = "https://heisenbug-premier-league-live-scores-v1.p.rapidapi.com/api/premierleague"

querystring = {"matchday":"28"}

headers = {
    'x-rapidapi-host': "heisenbug-premier-league-live-scores-v1.p.rapidapi.com",
    'x-rapidapi-key': "58ba862574msh2d22f18a5692fefp1e9a03jsn31da5fc820dc"
    }

matchday_matches = requests.request("GET", url, headers=headers,params=querystring)


In [7]:
matchday_matches = json.loads(matchday_matches.text)

In [8]:
matchday_matches

{'matches': [{'when': 'Wednesday, Jun 17 2020 18:00',
   'referee': 'Michael Oliver',
   'team1': {'teamName': 'Aston Villa', 'teamScore': 0, 'firstHalfScore': 0},
   'team2': {'teamName': 'Sheffield United',
    'teamScore': 0,
    'firstHalfScore': 0},
   'time': 'FT'},
  {'when': 'Saturday, Feb 29 2020 15:00',
   'referee': 'Andre Marriner',
   'team1': {'teamName': 'Bournemouth', 'teamScore': 2, 'firstHalfScore': 0},
   'team2': {'teamName': 'Chelsea', 'teamScore': 2, 'firstHalfScore': 1},
   'time': 'FT',
   'venue': 'Vitality Stadium',
   'attendance': ''},
  {'when': 'Saturday, Feb 29 2020 12:30',
   'referee': 'Martin Atkinson',
   'team1': {'teamName': 'Brighton', 'teamScore': 0, 'firstHalfScore': 0},
   'team2': {'teamName': 'Crystal Palace',
    'teamScore': 1,
    'firstHalfScore': 0},
   'time': 'FT',
   'venue': 'AMEX Stadium',
   'attendance': '30.124'},
  {'when': 'Sunday, Mar 01 2020 14:00',
   'team1': {'teamName': 'Everton', 'teamScore': 1, 'firstHalfScore': 1},
   '

In [9]:
all_matches = {}
i = 0

for matchday in range(1,max_matches+1):
    querystring = {"matchday":str(matchday)}
    matchday_matches = requests.request("GET", url, headers=headers,params=querystring)
    matchday_matches = json.loads(matchday_matches.text)
    for match in matchday_matches["matches"]:
        if 'time' in match and match['time'] == 'FT':
            all_matches[i] = [match['team1']['teamName'],match['team1']['teamScore'],match['team2']['teamName'],match['team2']['teamScore'],match['when']]
            i+=1
        

In [10]:
all_matches

{0: ['Liverpool', 4, 'Norwich', 1, 'Friday, Aug 09 2019 20:00'],
 1: ['West Ham', 0, 'Manchester City', 5, 'Saturday, Aug 10 2019 12:30'],
 2: ['Bournemouth', 1, 'Sheffield United', 1, 'Saturday, Aug 10 2019 15:00'],
 3: ['Burnley', 3, 'Southampton', 0, 'Saturday, Aug 10 2019 15:00'],
 4: ['Crystal Palace', 0, 'Everton', 0, 'Saturday, Aug 10 2019 15:00'],
 5: ['Leicester',
  0,
  'Wolverhampton Wanderers',
  0,
  'Sunday, Aug 11 2019 14:00'],
 6: ['Watford', 0, 'Brighton', 3, 'Saturday, Aug 10 2019 15:00'],
 7: ['Tottenham', 3, 'Aston Villa', 1, 'Saturday, Aug 10 2019 17:30'],
 8: ['Newcastle United', 0, 'Arsenal', 1, 'Sunday, Aug 11 2019 14:00'],
 9: ['Manchester United', 4, 'Chelsea', 0, 'Sunday, Aug 11 2019 16:30'],
 10: ['Arsenal', 2, 'Burnley', 1, 'Saturday, Aug 17 2019 12:30'],
 11: ['Aston Villa', 1, 'Bournemouth', 2, 'Saturday, Aug 17 2019 15:00'],
 12: ['Brighton', 1, 'West Ham', 1, 'Saturday, Aug 17 2019 15:00'],
 13: ['Chelsea', 1, 'Leicester', 1, 'Sunday, Aug 18 2019 16:30'

In [11]:
connection = pymysql.connect(host='fantasyfootballaws.cozgqt20qvfr.us-east-1.rds.amazonaws.com',
                             user='admin',
                             password='password',
                             db='fantasy_football',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

In [13]:
# db_data = 'mysql+mysqldb://' + 'admin' + ':' + 'password' + '@' + 'fantasyfootballaws.cozgqt20qvfr.us-east-1.rds.amazonaws.com' + ':3306/' \
#        + 'fantasy_football' + '?charset=utf8mb4'
# engine = create_engine(db_data)

In [15]:
# # create cursor
# cursor=connection.cursor()
# # Execute the to_sql for writting DF into SQL
# all_matches.to_sql('epl_fixtures', engine, if_exists='replace', index=False)   

AttributeError: 'dict' object has no attribute 'to_sql'

In [12]:

sql = "INSERT INTO `epl_fixtures` (`id`, `date`,`home_team`, `home_team_score`,`away_team`, `away_team_score`) VALUES (%s, %s,%s, %s,%s,%s)"

for i in all_matches:
#     cursor = connection.cursor()
    home_team = all_matches[i][0]
    home_team_score = all_matches[i][1]
    away_team = all_matches[i][2]
    away_team_score = all_matches[i][3]
    date = all_matches[i][4]
    with connection.cursor() as cursor:
        cursor.execute(sql,(i, str(date),home_team,home_team_score,away_team,away_team_score))
        connection.commit()
    

IntegrityError: (1062, "Duplicate entry '0' for key 'PRIMARY'")

-------------------------------

Now that I have a list of all of the matches played I will use this to get the player data from each of these matches

In [30]:
league_table

{'records': [{'team': 'Liverpool',
   'played': 29,
   'win': 27,
   'draw': 1,
   'loss': 1,
   'goalsFor': 66,
   'goalsAgainst': 21,
   'points': 82},
  {'team': 'Manchester City',
   'played': 28,
   'win': 18,
   'draw': 3,
   'loss': 7,
   'goalsFor': 68,
   'goalsAgainst': 31,
   'points': 57},
  {'team': 'Leicester',
   'played': 29,
   'win': 16,
   'draw': 5,
   'loss': 8,
   'goalsFor': 58,
   'goalsAgainst': 28,
   'points': 53},
  {'team': 'Chelsea',
   'played': 29,
   'win': 14,
   'draw': 6,
   'loss': 9,
   'goalsFor': 51,
   'goalsAgainst': 39,
   'points': 48},
  {'team': 'Manchester United',
   'played': 29,
   'win': 12,
   'draw': 9,
   'loss': 8,
   'goalsFor': 44,
   'goalsAgainst': 30,
   'points': 45},
  {'team': 'Sheffield United',
   'played': 28,
   'win': 11,
   'draw': 10,
   'loss': 7,
   'goalsFor': 30,
   'goalsAgainst': 25,
   'points': 43},
  {'team': 'Wolverhampton Wanderers',
   'played': 29,
   'win': 10,
   'draw': 13,
   'loss': 6,
   'goalsFor'

In [31]:
all_player_df = pd.DataFrame()

In [32]:
teams = []

for record in league_table['records']:
    teams.append(record['team'])

In [33]:
teams

['Liverpool',
 'Manchester City',
 'Leicester',
 'Chelsea',
 'Manchester United',
 'Sheffield United',
 'Wolverhampton Wanderers',
 'Tottenham',
 'Arsenal',
 'Burnley',
 'Crystal Palace',
 'Everton',
 'Newcastle United',
 'Southampton',
 'Brighton',
 'West Ham',
 'Bournemouth',
 'Watford',
 'Aston Villa',
 'Norwich']

--------------------

I will use the list of matches to get every player that played in each of those matches. Since goalkeepers, defenders, midfielders and forwards all have different stats available I will create one dataframe for each

The idea: We want to incentivise people to pick natural teams. Diminishing returns for getting too much of a certain statistic. Penalised for not getting enough of a certain stat. 

Encourages transfers because people will need to balance out their team

In [17]:
url = "https://heisenbug-premier-league-live-scores-v1.p.rapidapi.com/api/premierleague/formations"

querystring = {"team1":"Everton","team2":"Manchester United"}

headers = {
    'x-rapidapi-host': "heisenbug-premier-league-live-scores-v1.p.rapidapi.com",
    'x-rapidapi-key': "58ba862574msh2d22f18a5692fefp1e9a03jsn31da5fc820dc"
    }

response = json.loads(requests.request("GET", url, headers=headers, params=querystring).text)


In [18]:
response

{'team1': {'teamName': 'Everton',
  'players': {'lineups': [{'playerNumber': '1',
     'playerName': 'Jordan Pickford'},
    {'playerNumber': '23', 'playerName': 'Séamus Coleman', 'isCaptain': True},
    {'playerNumber': '3', 'playerName': 'Leighton Baines'},
    {'playerNumber': '26', 'playerName': 'Tom Davies'},
    {'playerNumber': '5', 'playerName': 'Michael Keane'},
    {'playerNumber': '2', 'playerName': 'Mason Holgate'},
    {'playerNumber': '11', 'playerName': 'Theo Walcott'},
    {'playerNumber': '21', 'playerName': 'André Gomes'},
    {'playerNumber': '7', 'playerName': 'Richarlison'},
    {'playerNumber': '9', 'playerName': 'Dominic Calvert-Lewin'},
    {'playerNumber': '10', 'playerName': 'Gylfi Sigurdsson'}],
   'substitutes': [{'playerNumber': '8', 'playerName': 'Fabian Delph'},
    {'playerNumber': '13', 'playerName': 'Yerry Mina'},
    {'playerNumber': '17', 'playerName': 'Alex Iwobi'},
    {'playerNumber': '19', 'playerName': 'Djibril Sidibé'},
    {'playerNumber': '20

In [19]:
for i in response['team1']['players']['substitutes']:
    print(i['playerName'])

Fabian Delph
Yerry Mina
Alex Iwobi
Djibril Sidibé
Bernard
Moise Kean
João Virgínia


In [22]:
# players_match_df = pd.DataFrame({'player_name': [], 'player_team': [],'starting_11': [],'match_id': []})

In [23]:
# url = "https://heisenbug-premier-league-live-scores-v1.p.rapidapi.com/api/premierleague/formations"

# headers = {
#     'x-rapidapi-host': "heisenbug-premier-league-live-scores-v1.p.rapidapi.com",
#     'x-rapidapi-key': "58ba862574msh2d22f18a5692fefp1e9a03jsn31da5fc820dc"
#     }

# for j in all_matches:
#     print(j)
#     team_1= all_matches[j][0]
#     team_1_score = all_matches[j][1]
#     team_2 = all_matches[j][2]
#     team_2_score = all_matches[j][3]
#     querystring = {"team1":team_1,"team2":team_2}
#     response = json.loads(requests.request("GET", url, headers=headers, params=querystring).text)
#     for i in response['team1']['players']['lineups']:
#         players_match_df = players_match_df.append({'player_name': i['playerName'], 'player_team':team_1,'starting_11': True, 'match_id':j},ignore_index=True)
#     for i in response['team1']['players']['substitutes']:
#         players_match_df = players_match_df.append({'player_name': i['playerName'], 'player_team':team_1,'starting_11': False,'match_id':j},ignore_index=True)
#     for i in response['team2']['players']['lineups']:
#         players_match_df = players_match_df.append({'player_name': i['playerName'], 'player_team':team_2,'starting_11': True,'match_id':j},ignore_index=True)
#     for i in response['team1']['players']['substitutes']:
#         players_match_df = players_match_df.append({'player_name': i['playerName'], 'player_team':team_2,'starting_11': False,'match_id':j},ignore_index=True)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [30]:
players_match_df = pd.DataFrame({'player_name': [], 'player_team': [],'team_1':[],'team_1_score':[],'team_2': [],'team_2_score': [],'starting_11': []})

In [31]:
for i in all_matches:
    print(i)
    team_1= all_matches[i][0]
    team_1_score = all_matches[i][1]
    team_2 = all_matches[i][2]
    team_2_score = all_matches[i][3]
    querystring = {"team1":team_1,"team2":team_2}
    response = json.loads(requests.request("GET", url, headers=headers, params=querystring).text)
    for i in response['team1']['players']['lineups']:
        players_match_df = players_match_df.append({'team_1':team_1,'team_2': team_2,'player_name': i['playerName'], 'player_team':team_1,'starting_11': True,'team_1_score':team_1_score,'team_2_score':team_2_score},ignore_index=True)
    for i in response['team1']['players']['substitutes']:
        players_match_df = players_match_df.append({'team_1':team_1,'team_2': team_2,'player_name': i['playerName'], 'player_team':team_1,'starting_11': False,'team_1_score':team_1_score,'team_2_score':team_2_score},ignore_index=True)
    for i in response['team2']['players']['lineups']:
        players_match_df = players_match_df.append({'team_1':team_1,'team_2': team_2,'player_name': i['playerName'], 'player_team':team_2,'starting_11': True,'team_1_score':team_1_score,'team_2_score':team_2_score},ignore_index=True)
    for i in response['team1']['players']['substitutes']:
        players_match_df = players_match_df.append({'team_1':team_1,'team_2': team_2,'player_name': i['playerName'], 'player_team':team_2,'starting_11': False,'team_1_score':team_1_score,'team_2_score':team_2_score},ignore_index=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [32]:
players_match_df

,player_name,player_team,team_1,team_1_score,team_2,team_2_score,starting_11
0,Alisson,Liverpool,Liverpool,4.0,Norwich,1.0,1.0
1,Trent Alexander-Arnold,Liverpool,Liverpool,4.0,Norwich,1.0,1.0
2,Andrew Robertson,Liverpool,Liverpool,4.0,Norwich,1.0,1.0
3,Fabinho,Liverpool,Liverpool,4.0,Norwich,1.0,1.0
4,Joseph Gomez,Liverpool,Liverpool,4.0,Norwich,1.0,1.0
...,...,...,...,...,...,...,...
13547,John Ruddy,Crystal Palace,Wolverhampton Wanderers,2.0,Crystal Palace,0.0,0.0
13548,Romain Saïss,Crystal Palace,Wolverhampton Wanderers,2.0,Crystal Palace,0.0,0.0
13549,Rúben Vinagre,Crystal Palace,Wolverhampton Wanderers,2.0,Crystal Palace,0.0,0.0
13550,Maximilian Kilman,Crystal Palace,Wolverhampton Wanderers,2.0,Crystal Palace,0.0,0.0


In [36]:
db_data = 'mysql+mysqldb://' + 'admin' + ':' + 'password' + '@' + 'fantasyfootballaws.cozgqt20qvfr.us-east-1.rds.amazonaws.com' + ':3306/' \
       + 'fantasy_football' + '?charset=utf8mb4'
engine = create_engine(db_data)

In [37]:
cursor=connection.cursor()

In [38]:
players_match_df.to_sql('match_players', engine, if_exists='replace', index=False) 

In [14]:
all_players = []
player_team = []

url = "https://heisenbug-premier-league-live-scores-v1.p.rapidapi.com/api/premierleague/players"

headers = {
    'x-rapidapi-host': "heisenbug-premier-league-live-scores-v1.p.rapidapi.com",
    'x-rapidapi-key': "58ba862574msh2d22f18a5692fefp1e9a03jsn31da5fc820dc"
    }

for team in teams:
    querystring = {"team":team}
    team_players = json.loads(requests.request("GET", url, headers=headers,params=querystring).text)
    for player in team_players['players']:
        all_players.append(player['playerName'])
        player_team.append(team)

In [15]:
len(all_players)

525

In [16]:
all_player_df['player'] = all_players
all_player_df['team'] = player_team

I now have a list of every match and a dataframe with every player. I will use this to create a dataframe with every player in every match and their stats for the match

In [17]:
all_players_all_matches =  pd.DataFrame()

In [18]:
columns = []

In [19]:
# first I will pick a match at random to use for getting all of the statistics available

In [1]:
url = "https://heisenbug-premier-league-live-scores-v1.p.rapidapi.com/api/premierleague/match/player"

querystring = {"team1":"Everton","team2":"Manchester United","player":player}

headers = {
    'x-rapidapi-host': "heisenbug-premier-league-live-scores-v1.p.rapidapi.com",
    'x-rapidapi-key': "58ba862574msh2d22f18a5692fefp1e9a03jsn31da5fc820dc"
    }

response = requests.request("GET", url, headers=headers, params=querystring)


NameError: name 'player' is not defined

In [21]:
url = "https://heisenbug-premier-league-live-scores-v1.p.rapidapi.com/api/premierleague/match/player"

headers = {
    'x-rapidapi-host': "heisenbug-premier-league-live-scores-v1.p.rapidapi.com",
    'x-rapidapi-key': "58ba862574msh2d22f18a5692fefp1e9a03jsn31da5fc820dc"}

player_count = 0

for player in all_player_df[all_player_df['team'] == 'Manchester United']['player']:
    querystring = {"team1":"Everton","team2":"Manchester United","player":player}
    player_data = json.loads(requests.request("GET", url, headers=headers, params=querystring).text)
    print(player, ": ")
    print(player_data)
    print()
    if 'isManOfTheMatch' in player_data:
        player_count+= 1

Aaron Wan-Bissaka : 
{'isManOfTheMatch': False, 'shirtNo': '29', 'name': 'Aaron Wan-Bissaka', 'isFirstEleven': True, 'position': 'DR', 'stats': {'touches': 61, 'passesTotal': 35, 'passesSuccessful': 27, 'aerialsTotal': 3, 'aerialsWon': 1, 'tacklesTotal': 2, 'tackleSuccessful': 1, 'tackleUnsuccesful': 1, 'throwInsTotal': 12, 'clearances': 3, 'defensiveAerials': 3, 'dribbledPast': 1, 'dispossessed': 2, 'possessionMins': 3, 'crossesTotal': 4, 'in': 0, 'out': 90}}

Sergio Romero : 
{'isManOfTheMatch': False, 'shirtNo': '22', 'name': 'Sergio Romero', 'position': 'Sub', 'stats': {}}

Harry Maguire : 
{'isManOfTheMatch': False, 'shirtNo': '5', 'name': 'Harry Maguire', 'isFirstEleven': True, 'position': 'DC', 'stats': {'touches': 68, 'passesTotal': 44, 'passesSuccessful': 35, 'aerialsTotal': 7, 'aerialsWon': 5, 'interceptions': 2, 'clearances': 15, 'offensiveAerials': 2, 'defensiveAerials': 5, 'foulsCommited': 2, 'offsidesCaught': 1, 'possessionMins': 5, 'in': 0, 'out': 90}}

Eric Bailly : 
{'

In [22]:
player_count

17

In [23]:
url = "https://heisenbug-premier-league-live-scores-v1.p.rapidapi.com/api/premierleague/match/events"

querystring = {"team1":"Manchester United","team2":"Watford"}

headers = {
    'x-rapidapi-host': "heisenbug-premier-league-live-scores-v1.p.rapidapi.com",
    'x-rapidapi-key': "58ba862574msh2d22f18a5692fefp1e9a03jsn31da5fc820dc"
    }

response = requests.request("GET", url, headers=headers, params=querystring)


In [24]:
response = json.loads(response.text)

In [25]:
response

{'team1': {'teamName': 'Manchester United',
  'events': [{'minute': "0'", 'value': '4231', 'eventName': 'Module-Start'},
   {'minute': "40'",
    'players': [{'playerName': 'Bruno Fernandes'}],
    'eventName': 'Penalty-Won'},
   {'minute': "41'",
    'players': [{'playerName': 'Bruno Fernandes'}],
    'eventName': 'Goal'},
   {'minute': "58'",
    'players': [{'playerName': 'Anthony Martial'}],
    'eventName': 'Goal'},
   {'minute': "74'",
    'players': [{'playerName': 'Mason Greenwood'}],
    'eventName': 'Goal'},
   {'minute': "74'",
    'players': [{'playerName': 'Bruno Fernandes'}],
    'eventName': 'Assist-Goal'},
   {'minute': "79'",
    'players': [{'playerName': 'Fred'},
     {'playerName': 'Mason Greenwood'},
     {'playerName': 'Anthony Martial'}],
    'eventName': 'Out'},
   {'minute': "79'",
    'players': [{'playerName': 'Scott McTominay'},
     {'playerName': 'Tahith Chong'},
     {'playerName': 'Odion Ighalo'}],
    'eventName': 'In'}]},
 'team2': {'teamName': 'Watfor